# Demo ML application

In this notebook, we will predict whether a wine is high quality given its chemical attributes. This notebook depends on the pickle output of the "Model Build" notebook.

<img alt="Wine" src="img/wine.jpg"/>

### Launch the app

If the app is not running, the following will break!

In [1]:
import os
import requests

# if you have a proxy...
os.environ['NO_PROXY'] = 'localhost'

# test if it's running
url = "http://localhost:5000"
test_url = "%s/test" % url
pred_url = "%s/predict" % url

# print the GET result
print(requests.get(test_url).json()['message'])

App is live!


### Load the data

In [2]:
import pandas as pd

data = pd.read_csv('data/allwine.csv')
ground_truth = data.pop('quality')
print(data.shape)
data.head()

(5320, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,1.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,1.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1.0
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,1.0


## Generate predictions

Now that we have a live application, let's generate some predictions

In [3]:
predictions = requests.post(pred_url, json={'data': data.as_matrix().tolist()}).json()

# append predictions to the table
predicted = data.copy()
predicted['predicted_quality'] = predictions['predictions']
predicted.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red,predicted_quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1.0,5.089106
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,1.0,5.041541
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,1.0,5.385300
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1.0,5.893353
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,1.0,5.137886


## How can we use this?

It's not enough to simply build a good model. There has to be a valuable business problem to solve! What if we could recommend new wines on the basis of their predicted quality?

In [4]:
def recommend_top_k_wines(wines, k=10):
    return wines.sort_values('predicted_quality', 
                             ascending=False)

def recommend_top_k_reds(wines, k=10):
    return recommend_top_k_wines(wines.loc[wines.red == 1.0])

def recommend_top_k_whites(wines, k=10):
    return recommend_top_k_wines(wines.loc[wines.red == 0.0])

In [5]:
top_10_reds = recommend_top_k_reds(predicted, 10)
top_10_reds.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red,predicted_quality
952,7.9,0.54,0.34,2.5,0.076,8.0,17.0,0.99235,3.20,0.72,13.1,1.0,7.404547
235,7.9,0.35,0.46,3.6,0.078,15.0,37.0,0.99730,3.35,0.86,12.8,1.0,7.404226
397,11.3,0.62,0.67,5.2,0.086,6.0,19.0,0.99880,3.22,0.69,13.4,1.0,7.352399
242,10.3,0.32,0.45,6.4,0.073,5.0,13.0,0.99760,3.23,0.82,12.6,1.0,7.348074
510,5.0,0.42,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0,1.0,7.335660


In [6]:
top_10_whites = recommend_top_k_whites(predicted, 10)
top_10_whites.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red,predicted_quality
2053,7.4,0.24,0.36,2.00,0.031,27.0,139.0,0.99055,3.28,0.48,12.5,0.0,7.668062
2595,8.0,0.44,0.49,9.10,0.031,46.0,151.0,0.99260,3.16,0.27,12.7,0.0,7.655121
2619,7.7,0.35,0.49,8.65,0.033,42.0,186.0,0.99310,3.14,0.38,12.4,0.0,7.598987
2442,8.0,0.45,0.36,8.80,0.026,50.0,151.0,0.99270,3.07,0.25,12.7,0.0,7.596299
3962,6.9,0.36,0.35,8.60,0.038,37.0,125.0,0.99160,3.00,0.32,12.4,0.0,7.596213


### Theoretical usecase

Most modern recommender systems operate on the basis of user ratings (collaborative filtering), but if we have new products that have not been rated, how can we make recommendations? This is called the cold-start problem. However, if we can build a model to predict quality based on historical data's mapping of attributes to ratings, we can offer recommendations on the basis of highest predicted quality.

A vineyard or liquor store could adopt a model like this to drive sales!